## Свёрточные сети для классификации

In [1]:
from typing import Type, Optional

import torch
from torch import Tensor, nn
from torch.nn import functional as F

#### Задание 1. Skip-connections (2 балла)

Постройте архитектуру свёрточной сети, аналогичную архитектуре в примере ниже, но добавьте в неё skip-connections, то есть дополнительные рёбра в вычислительном графе, позволяющие пропускать градиент в более ранние слои напрямую, минуя очередной блок Conv2D + BatchNorm + ReLU:

```python
def forward(self, x: Tensor) -> Tensor:
    x = x + self.block1(x)
    x = self.maxpool(x)
    x = x + self.block2(x)
    x = self.maxpool(x)
    ...
    x = x.adaptive_maxpool(x).flatten(1)
    logits = self.fc(x)
    return logits
```


Наша верхнеуровневая архитектура будет выглядеть так:

In [2]:
class MyResNet(nn.Module):
    def __init__(
        self,
        block: Type[nn.Module],
        n_classes: int,
        hidden_channels: list[int] = [32, 64],
    ) -> None:
        super().__init__()
        # входной слой, принимающий изображение с 3-мя каналами
        self.in_conv = nn.Conv2d(3, hidden_channels[0], kernel_size=3, stride=1)
        self.relu = nn.ReLU(inplace=True)

        # собираем свёрточные блоки, каждый задаётся кол-вом входных и выходных каналов
        blocks = []
        for c_in, c_out in zip(hidden_channels[:-1], hidden_channels[1:]):
            # добавляем очередной блок
            blocks.append(block(c_in, c_out))
            # добавляем Max pooling для уменьшения размерности
            blocks.append(nn.MaxPool2d(2, 2))

        # собираем блоки в единый Sequential модуль для удобства
        self.features = nn.Sequential(*blocks)
        self.maxpool = nn.AdaptiveMaxPool2d(1)

        # линейный слой для классификации
        self.fc = nn.Linear(hidden_channels[-1], n_classes)

    def forward(self, x: Tensor) -> Tensor:
        h = self.features(self.relu(self.in_conv(x)))
        logits = self.fc(self.maxpool(h).flatten(1))
        return logits

Базовый блок, без residual connections, состоит из двух свёрток и нормализаций:

In [3]:
class BasicBlock(nn.Module):
    def __init__(self, inplanes: int, planes: int) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

    def forward(self, x: Tensor) -> Tensor:
        # first conv + bn + nonlinearity
        out = self.relu(self.bn1(self.conv1(x)))
        # second conv + bn
        out = self.bn2(self.conv2(out))
        # final nonlinearity
        out = self.relu(out)
        return out

Посмотрим на результат его применения к тензору:

In [4]:
BasicBlock(4, 6).forward(torch.randn(3, 4, 32, 32)).shape

torch.Size([3, 6, 32, 32])

Теперь нужно изменить этот блок, добавив в него skip-connection. Теперь в методе `forward` входной тензор `x` пойдёт по двум веткам:
1. как в базовом блоке, через наши всёртки и нормализации, до последней нелинейности
2. в обход свёрток и нормализаций

В конце эти ветки нужно объединить через сумму. Тут есть проблема: в исходном тензоре `x` и обработанном нашим блоком `h(x)` отличается количество каналов (остальные размерности совпадают). То есть нам нужно сравнять количество каналов исходного тензора `inplanes` с количеством выходных каналов `outplanes`.

Интуитивно, если рассматривать каждый пиксел входного тензора как вектор размера `inplanes`, в вектор размера `planes` его можно превратить домножением на матрицу размера `inplanes x planes`. Это можно сделать, создав свёрточный слой с размером кернела 1 - он и будет переводить наши пикселы в другую размерность.

Не забудьте к сумме каналов применить нелинейность.

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, inplanes: int, planes: int) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        # добавьте свёртку 1x1 для изменения кол-ва каналов входного тензора
        self.downsample = None
        self.downsample = nn.Conv2d(inplanes, planes, kernel_size=1, stride=1, bias=False, padding=0)

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        # ВАШ ХОД
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity

        out = self.relu(out)

        return out

Проверим размеры:

In [6]:
assert ResidualBlock(4, 6).forward(torch.randn(3, 4, 32, 32)).shape == torch.Size(
    [3, 6, 32, 32]
)

Проверим, что модель выдаёт тензор ожидаемого размера:

In [7]:
MyResNet(ResidualBlock, 7, hidden_channels=[16, 32, 64, 128]).forward(
    torch.randn(3, 3, 32, 32)
).shape

torch.Size([3, 7])

Теперь мы можем создавать модели разного размера, в том числе достаточно большие и глубокие, чтобы хорошо классифицировать изображения из датасета CIFAR-10.

In [8]:
sum(
    p.numel()
    for p in MyResNet(ResidualBlock, 7, hidden_channels=[16, 32, 64, 64]).parameters()
)

151047

#### Задание 2. Обучение `MyResNet` с использованием Lightning (5 баллов)

Ваша задача: добиться 80% точности на валидационной выборке с вашей реализацией `MyResNet`.

После окончания обучения используйте метод `Trainer.validate` для вывода ваших метрик с удачного чекпоинта модели.

NB: вызывайте `Trainer.validate` везде, где в задании требуется достичь какой-то точности


Советы:
- По умолчанию Lightning сохраняет только последний чекпоинт, так что вам может потребоваться `lightning.callbacks.ModelCheckpoint`, чтобы сохранять лучший чекпоинт в процессе обучения.

- Используйте tensorboard, чтобы следить за динамикой обучения. Если заметите переобучение - подключайте регуляризацию. Большая модель с регуляризацией обычно лучше маленькой модели без неё.

- Чтобы добиться нужной точности, ваша модель должна быть достаточно глубокой, ориентируйтесь на 4-5 блоков. Если необходимо, подключайте регуляризацию

In [11]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
import torch
import torchmetrics
from torch.utils.data import DataLoader
from torch.nn import functional as F

import lightning as L
from lightning.pytorch.utilities.types import EVAL_DATALOADERS, TRAIN_DATALOADERS
from torchvision import datasets, transforms
from typing import Callable, Any
from PIL.Image import Image
from torch import Tensor, nn



class Datamodule(L.LightningDataModule):
    def __init__(
        self,
        batch_size: int,
        transform: Callable[[Image], Tensor] = transforms.ToTensor(),
        num_workers: int = 0,
    ):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transform
        self.num_workers = num_workers

    def prepare_data(self) -> None:
        # в этом методе можно сделать предварительную работу, например
        # скачать данные, сделать тяжёлый препроцессинг
        pass

    def setup(self, stage: str) -> None:
        # аргумент `stage` будет приходить из модуля обучения Trainer
        # на стадии обучения (fit) нам нужны оба датасета
        if stage == "fit":
            self.train_dataset = datasets.CIFAR10(
                "data",
                train=True,
                download=True,
                transform=self.transform,
            )
            self.val_dataset = datasets.CIFAR10(
                "data",
                train=False,
                download=True,
                transform=transforms.ToTensor(),
            )
        # на стадии валидации (validate) - только тестовый
        elif stage == "validate":
            self.val_dataset = datasets.CIFAR10(
                "data",
                train=False,
                download=True,
                transform=transforms.ToTensor(),
            )
        else:
            raise NotImplementedError
        # есть ещё стадии `test` и `predict`, но они нам не понадобятся

    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )
    

datamodule = Datamodule(batch_size=128)
class LitModel(L.LightningModule):
    def __init__(self, model: nn.Module, learning_rate: float = 1e-3):
        super().__init__()
        self.save_hyperparameters(ignore=['model'])
        self.model = model
        self.learning_rate = learning_rate

    def forward(self, x: Tensor) -> Tensor:
        return self.model(x)

    def training_step(self, batch: tuple[Tensor, Tensor], batch_idx: int) -> Tensor:
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()

        self.log("train_loss", loss, on_step=False, on_epoch=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch: tuple[Tensor, Tensor], batch_idx: int) -> None:
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()

        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def configure_optimizers(self) -> dict[str, Any]:
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[5, 10, 15], gamma=0.5
        )
        return {"optimizer": optimizer, "lr_scheduler": scheduler}

model = MyResNet(
    ResidualBlock,
    n_classes=10,
    hidden_channels=[16, 32, 64, 64],
)

lit_model = LitModel(model)

logger = TensorBoardLogger("tb_logs", name="MyResNet")

checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    mode='max',
    save_top_k=1,
    filename='best-checkpoint',
    verbose=True,
)

trainer = L.Trainer(
    accelerator='auto',
    devices='auto',
    max_epochs=20,
    logger=logger,
    callbacks=[checkpoint_callback],
    #limit_train_batches=999,
    #limit_val_batches=999,
)

trainer.fit(
    model=lit_model,
    datamodule=datamodule,
)
trainer.validate(
    model=lit_model,
    datamodule=datamodule,
    ckpt_path=checkpoint_callback.best_model_path,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type     | Params | Mode 
-------------------------------------------
0 | model | MyResNet | 151 K  | train
-------------------------------------------
151 K     Trainable params
0         Non-trainable params
151 K     Total params
0.605     Total estimated model params size (MB)
30        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Files already downloaded and verified


Restoring states from the checkpoint path at tb_logs/MyResNet/version_12/checkpoints/best-checkpoint.ckpt
Loaded model weights from the checkpoint at tb_logs/MyResNet/version_12/checkpoints/best-checkpoint.ckpt


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.8374999761581421
        val_loss             0.512255847454071
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.512255847454071, 'val_acc': 0.8374999761581421}]

#### Задание 3. Добавление аугментаций (1 балл + 2 балла за точность на валидации более 85%)

Добавьте к обучающему датасету аугментации - случайные трансформации входных данных. Для этого можно использовать `torchvision.transforms` и `albumentations`.

С `torchvision.transforms` совсем просто: вам нужно будет при создании `Datamodule` из практики по `lightning` указать вместо

```python
transform = transforms.ToTensor()
```
композицию трансформаций:

```python
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # случайное зеркальное отражение
    ...
    transforms.ToTensor(),
])
```

В пакете `albumentations` аугментаций значительно больше:

![albumentations](https://albumentations.ai/assets/img/custom/top_image.jpg)

#### Задание 4. Использование предобученной модели (4 балла)

Теперь мы научимся использовать модели, обученные на других задачах

Ваша задача: добиться 90% точности на тестовой выборке CIFAR-10. Постарайтесь уложиться модель с ~5 млн параметров

В `torchvision.models` есть много реализованных архитектур, размером которых можно удобно управлять. Например, ниже можно создать крошечную версию модели `MobileNetV2`:

In [18]:
from torchvision.models import MobileNetV2

mobilenet = MobileNetV2(
    num_classes=10,
    width_mult=0.4,
    inverted_residual_setting=[
        # t, c, n, s
        [1, 16, 1, 1],
        [3, 24, 2, 2],
        [3, 32, 3, 2],
    ],
    dropout=0.2,
)

sum([param.numel() for param in mobilenet.parameters()])

46322

Но кроме архитектуры модели, мы также можем скачать веса, полученные при обучении на каком-то датасете. Например, для нашей задачи можно использовать предобучение на самом известном датасете для классификации изображений - ImageNet:

In [19]:
from torchvision.models.efficientnet import EfficientNet_B0_Weights, efficientnet_b0

# создаём EfficientNet с весами, полученными на ImageNet
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
efficientnet = efficientnet_b0(weights=weights)
sum([param.numel() for param in efficientnet.parameters()])

5288548

**Указание 1.** С использованием модели в исходном виде есть проблема: в ImageNet 1000 классов, а у нас только 10. Поэтому в предобученной модели нужно будет полностью заменить последний линейный слой, который даёт распределение вероятностей классов. Это можно сделать уже в готовом объекте модели, переназначив атрибут.

Подсказка: в `efficientnet_b0` линейный слой находится в атрибуте `classifier` 


**Указание 2.** Все слои, кроме нескольких последних (может быть, только последнего) мы можем заморозить, то есть сделать значения параметров в них неизменными. Это позволит и сохранить способность модели выделять полезные низкоуровневые признаки (она научилась этому на ImageNet), и существенно ускорить дообучение.


Чтобы заморозить параметры, нужно всего лишь отключить для них расчёт градиентов. Вернитесь к первой практике, чтобы вспомнить, как это можно сделать. Нам подойдёт самый простой способ с `.requires_grad`.

Подсказка: в `efficientnet_b0` свёрточные слои находятся в атрибуте `features` 

**Указание 3.** Предобученные модели на ImageNet ожидают специальным образом трансформированные изображения:


In [25]:
weights.transforms()

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

Поэтому эти трансформации нужно будет передать в датамодуль (как мы делали с аугментациями).

ВАШ ХОД: Обучите модель и выведите результат метода validate на удачном чекпоинте